<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/Word2Vec_sample001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec sample

参考「15分でできる日本語Word2Vec」：https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

## ①三四郎をダウンロードし、データを成型する

In [ ]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = '794_ruby_4237.zip'
import urllib.request
urllib.request.urlretrieve(url, zip)

# ダウンロードしたzipの解凍
import zipfile
with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




## ②Janomeを用いて三四郎テキストから単語リストを生成

In [ ]:
# Janomeのインストール
# !pip install janome | tail -n 1 
# ↑tailはMacで利用できるログを出力するコマンド。Windowsでは利用できないがログを出力しなくても問題ないのでjanomeのインストールのみ行う。

!pip install janome

# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

     |████████████████████████████████| 19.7MB 1.2MB/s 
一
する
目
さめる
女
隣
じいさん
話
始める
いる


## ③準備したデータを用いてWord2Vecのモデル作成、学習実施

In [ ]:
# Word2Vecライブラリの導入
!pip install gensim  

# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

## 結果確認①　単語のベクトルの値を確認する

In [ ]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])

[-0.09137168 -0.18239665 -0.14266613 -0.03590868  0.4106189   0.05194617
  0.42041898 -0.26192182  0.92724633  0.2463157   0.28622583  0.10278496
 -0.33419138  0.44035092 -0.8152608  -0.17599252  0.04053064 -0.24757783
  0.26843548  0.64016914  0.5903478   0.93825805  0.8168316   0.73487306
 -0.27847102  0.7643608   0.42424157 -1.0827986   0.06014311 -0.05226474
  0.43764862  0.4893643   0.14749189 -0.34626526 -0.36274204  0.62168676
 -0.18103878 -0.21117617  0.24598132 -0.64972734 -0.73778486 -0.3418978
 -0.15678014  0.11407202  0.27941033  0.1173316   0.71309155 -0.6858391
 -0.1752132  -0.20411023 -0.5762656  -0.12559824 -0.13221966 -0.48963836
 -0.13745916 -0.46204978 -0.71786326  0.01657034 -0.21952264 -0.88240564
  0.24847524 -0.17602473  0.21283443 -0.28358278  0.24748263 -0.40335053
  0.31509393 -0.7813227   0.01896676 -0.10138191  0.0213471  -0.08959553
  0.19846459 -0.05011254 -0.02921634 -0.5591167   0.12708788 -0.10888859
 -0.14537685 -0.04439389  0.09432849 -0.08758259  0.3

## 結果確認②　単語のベクトルから類似単語を推定する。

In [ ]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

聞こえる 0.5856649279594421
外国 0.5547837018966675
自己 0.5524786710739136
社会 0.4986944794654846
喝采 0.4957362115383148
堪える 0.4861728549003601
くせ 0.47196945548057556
今日 0.46249067783355713
文学 0.45822232961654663
決心 0.45598113536834717


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
